Input data

In [ ]:
# Input and output folders
input_folder = r"D:\River connectivity\Whole Danube Reach in Hungary\Water mask output\clear mask-use this"  # original water masks path 
output_folder = "D:/River connectivity/Whole Danube Reach in Hungary/pre-processing-watermask-input for qgis-CL extraction"  # enhanced water mask path 
prefix = "enhanced_"
# the kernal sizes of smoothing and filling gaps in steps 2, optinal and 3 can be finetuned  

Processing code

In [ ]:
import os
import rasterio
import numpy as np
import cv2
from scipy.ndimage import binary_fill_holes, binary_closing

# Ensure the output directory exists
os.makedirs(output_folder, exist_ok=True)

# Process each tif file in the input folder
for file_name in os.listdir(input_folder):
    if file_name.endswith(".tif"):
        input_path = os.path.join(input_folder, file_name)
        output_path = os.path.join(output_folder, prefix + file_name)
        
        # Load water mask from the tif image
        with rasterio.open(input_path) as src:
            water_mask = src.read(1)  # Read the first band
            output_meta = src.meta.copy()

        # Convert to binary mask (assuming water is non-zero and background is zero)
        binary_mask = (water_mask > 0).astype(np.uint8)

        # 1. Fill holes caused by ships or small islands
        filled_mask = binary_fill_holes(binary_mask)

        # 2. Smooth the river edges
        smoothing_kernel_size = 3
        smoothing_kernel = np.ones((smoothing_kernel_size, smoothing_kernel_size), np.uint8)
        smoothed_mask = cv2.morphologyEx(filled_mask.astype(np.uint8), cv2.MORPH_CLOSE, smoothing_kernel)

        # Optional: Apply another dilation to smooth further if necessary
        extra_smoothing_kernel_size = 3
        extra_smoothing_kernel = np.ones((extra_smoothing_kernel_size, extra_smoothing_kernel_size), np.uint8)
        smoothed_mask = cv2.dilate(smoothed_mask, extra_smoothing_kernel, iterations=1)

        # 3. Fill gaps
        gap_filling_kernel_size = 7
        gap_filling_kernel = np.ones((gap_filling_kernel_size, gap_filling_kernel_size), np.uint8)
        filled_gaps_mask = binary_closing(smoothed_mask, structure=gap_filling_kernel)

        # 4. Ensure we don't break existing river structure
        final_mask = np.logical_or(binary_mask, filled_gaps_mask).astype(np.uint8)

        # Save the processed water mask with updated metadata
        output_meta.update({"dtype": 'uint8'})
        with rasterio.open(output_path, 'w', **output_meta) as dst:
            dst.write(final_mask, 1)

        print(f"Processing complete. Processed water mask saved as '{output_path}'.")
